In [1]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from vehi_rout.utils.helper_utils import get_values_not_in_second_list
from vehi_rout.utils.visualization import visualize_routes_per_vehicle
from vehi_rout.utils.data_utils import get_demand_df, update_demand_dic, load_matrix_df, get_demand_matrix_df, load_df
from vehi_rout.utils.route_utils import get_penalty_list, sort_nodes_by_distance
from vehi_rout.constant import *

load c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\ortools.dll...
Loaded 1361 cached routes from ../data/csv/route_cache.csv


In [2]:
TOTAL_DAYS = 6
MAX_VISITS_PER_VEHICLE = [10]*2 + [20]*3 #+ [20]*2

TIME_BASE_PENALTY = 100000
MAX_TIME_PER_VEHICLE = [1200, 1200, 600, 600, 600, 600, 600, 600]

DISTANCE_BASE_PENALTY = 1000
MAX_DISTANCE_PER_VEHICLE = [400]*2 + [900]*3 #+ [1000]*2

DEPOT = 0

In [3]:
def print_solution(manager, routing, solution, data, day):
    total_distance = 0  # Changed from total_time to total_distance
    visited_nodes = set()
    route_dict = {}  # Dictionary to store route details for each vehicle
    
    print(f"\nDay {day + 1} Routes (Penalty per unvisited demand unit: {data['penalties'][1]} units):")
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0  # Changed from route_time to route_distance
        num_visits = 0
        route_nodes = []  # List to store the sequence of nodes for this vehicle
        previous_node = None
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            original_node = data["node_mapping"][node]
            visited_nodes.add(original_node)
            route_nodes.append(original_node)
            plan_output += f" {original_node} ->"
            if previous_node is not None:
                arc_distance = int(data["distance_matrix"][previous_node][node])  # Changed from time_matrix to distance_matrix
                route_distance += arc_distance
            previous_node = node
            index = solution.Value(routing.NextVar(index))
            if original_node != data["depot"]:
                num_visits += 1
        
        node = manager.IndexToNode(index)
        original_node = data["node_mapping"][node]
        visited_nodes.add(original_node)
        route_nodes.append(original_node)
        plan_output += f" {original_node}\n"
        if previous_node is not None:
            arc_distance = int(data["distance_matrix"][previous_node][node])  # Changed from time_matrix to distance_matrix
            route_distance += arc_distance
        
        plan_output += f"Distance of the route: {route_distance} km\n"  # Changed from Time to Distance, assumed km
        max_distance = data["max_distance_per_vehicle"][vehicle_id]  # Changed from max_time_per_vehicle to max_distance_per_vehicle
        plan_output += f"Within limit: {'Yes' if route_distance <= max_distance else 'No'} (Max: {max_distance} km)\n"  # Changed units to km
        plan_output += f"Stops visited: {num_visits-1}/{data['max_visits_per_vehicle'][vehicle_id]}\n"
        
        # Store route details in the dictionary
        route_dict[vehicle_id] = {
            "route_nodes": route_nodes,
            "route_distance": route_distance,  # Changed from route_time to route_distance
            "max_distance_limit": max_distance,  # Changed from max_time_limit to max_distance_limit
            "within_limit": route_distance <= max_distance,
            "num_visits": num_visits,
            "max_visits_limit": data["max_visits_per_vehicle"][vehicle_id]
        }
        
        print(plan_output)
        total_distance = max(total_distance, route_distance)  # Changed from total_time to total_distance
    
    print(f"Maximum route distance for Day {day + 1}: {total_distance} km")  # Changed from time to distance
    
    return visited_nodes, route_dict  # Return both visited_nodes and route_dict

In [4]:
# In data_model.vrp_data_model.py
def create_data_model(full_matrix, nodes_to_visit, demand_dict, penalty_lis, use_distance=False):
    data = {}
    node_indices = [0] + [i for i, code in enumerate(full_matrix.index) if code in demand_dict['key']]
    nodes_to_use = [node_indices[0]] + [i for i in node_indices[1:] if i in nodes_to_visit]
    data["num_vehicles"] = len(MAX_DISTANCE_PER_VEHICLE)
    data["depot"] = 0
    if use_distance:
        # Construct distance matrix from full_matrix
        data["distance_matrix"] = [[full_matrix.iloc[i][j] for j in nodes_to_use] for i in nodes_to_use]
        data["max_distance_per_vehicle"] = MAX_DISTANCE_PER_VEHICLE  
    else:
        # Original time matrix (for backward compatibility)
        data["time_matrix"] = [[full_matrix.iloc[i][j] for j in nodes_to_use] for i in nodes_to_use]
        data["max_time_per_vehicle"] = MAX_TIME_PER_VEHICLE 
        
    data["demands"] = [0] + [demand_dict.get(full_matrix.index[i], 1) for i in nodes_to_use[1:]]  # Demand at each node
    data["node_mapping"] = [full_matrix.index[i] for i in nodes_to_use] #{i: node for i, node in enumerate([0] + nodes_to_visit)}  # Mapping of indices to nodes
    data["max_visits_per_vehicle"] = MAX_VISITS_PER_VEHICLE 
    data["penalties"] = [0] + penalty_lis
    return data

In [5]:
def solve_vrp_for_day(full_matrix, nodes_to_visit, day, demand_dict, penalty_lis):
    # Assuming create_data_model now returns a distance_matrix instead of time_matrix
    data = create_data_model(full_matrix, nodes_to_visit, demand_dict,penalty_lis, use_distance=True)  # Adjust create_data_model
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = int(data["distance_matrix"][from_node][to_node])  # Use distance instead of time
        return distance

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Distance dimension (replacing Time dimension)
    routing.AddDimension(
        transit_callback_index,
        0,
        max(data["max_distance_per_vehicle"]),  # New max distance limit per vehicle
        True,
        "Distance"
    )
    distance_dimension = routing.GetDimensionOrDie("Distance")
    for vehicle_id in range(data["num_vehicles"]):
        end_index = routing.End(vehicle_id)
        distance_dimension.CumulVar(end_index).SetMax(data["max_distance_per_vehicle"][vehicle_id])  # Set distance limit

    # Demand dimension remains unchanged
    def demand_callback(from_index):
        return data["demands"][manager.IndexToNode(from_index)]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,
        data["max_visits_per_vehicle"],
        True,
        "Visits"
    )

    # Optional nodes with demand-based penalty (unchanged)
    for node in range(1, len(data["distance_matrix"])):
        routing.AddDisjunction([manager.NodeToIndex(node)], data["penalties"][node])

    # Search parameters (unchanged)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 30

    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        return print_solution(manager, routing, solution, data, day)
    else:
        print(f"No solution found for Day {day + 1}!")
        return set()

In [6]:
demand_df = get_demand_df(
     today_path='../data/orders/03-03-2025-PO.csv',
    #  wait_path='../data/orders/2/next_day_demand.csv'
    )

demand_df.shape

(454, 8)

In [7]:
if demand_df['CODE'].dtype in ['float', 'int', 'int64']:
    demand_df['CODE'] = demand_df['CODE'].astype(int)
    demand_df['CODE'] = demand_df['CODE'].astype(str)
    print('Converting to Object')

In [8]:
master_mat_df = load_matrix_df(path='../data/master/osrm_distance_matrix.csv')

In [9]:
demand_mat_df = get_demand_matrix_df(master_mat_df,demand_df, 0)

In [10]:
demand_mat_df.shape

(455, 455)

In [11]:
demand_dict = update_demand_dic(demand_df)

In [12]:
len(demand_dict['key'])

454

In [13]:
len(demand_dict['po_date'])

454

In [14]:
today = '2025-03-03'
penalty_lis = get_penalty_list(demand_dict, DISTANCE_BASE_PENALTY, TOTAL_DAYS, today)

In [15]:
demand_sort_nodes = sort_nodes_by_distance(demand_mat_df.values)

In [16]:
nodes_to_visit = sorted(demand_sort_nodes[:300])

In [17]:
visited_node, route_dict = solve_vrp_for_day(demand_mat_df, nodes_to_visit, 1, demand_dict, penalty_lis)

C:\Users\HP\AppData\Local\Temp\ipykernel_18020\1512498709.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data["distance_matrix"] = [[full_matrix.iloc[i][j] for j in nodes_to_use] for i in nodes_to_use]



Day 2 Routes (Penalty per unvisited demand unit: 100 units):
Route for vehicle 0:
 0 -> SGRA -> SGMH -> 150 -> SGEL -> SGMP -> SGGP -> 24 -> 114 -> SGYK -> SGK5 -> 0
Distance of the route: 42 km
Within limit: Yes (Max: 400 km)
Stops visited: 10/10

Route for vehicle 1:
 0 -> SGEM -> 3 -> 117 -> 14015 -> SGWA -> SGKR -> SGTY -> 14 -> SGKB -> SGKG -> 0
Distance of the route: 14 km
Within limit: Yes (Max: 400 km)
Stops visited: 10/10

Route for vehicle 2:
 0 -> SCUP -> 5 -> 1017 -> 1005 -> 1818 -> 1108 -> SCKE -> SCK7 -> 1137 -> SCMY -> SGNO -> 128 -> SGSE -> SGKC -> 1084 -> SGJA -> 171 -> 14013 -> SGJW -> SGGE -> 0
Distance of the route: 72 km
Within limit: Yes (Max: 900 km)
Stops visited: 20/20

Route for vehicle 3:
 0 -> SCKX -> 1678 -> 14010 -> 110 -> SLMN -> SCBI -> SCTH -> SCPT -> SCAR -> SCTG -> SCBT -> SCPK -> SCTL -> SCHD -> SCVJ -> SCAK -> 1549 -> 1548 -> 1355 -> SGMQ -> 0
Distance of the route: 52 km
Within limit: Yes (Max: 900 km)
Stops visited: 20/20

Route for vehicle 4:
 0

In [400]:
not_visited_codes = get_values_not_in_second_list(demand_df['CODE'], list(visited_node))

not_visited_codes_str = list(map(str, not_visited_codes))
# Filter the dataframe
sub_df = demand_df[demand_df['CODE'].isin(not_visited_codes_str)]
print(sub_df.shape)
# sub_df.to_csv('../data/orders/2/next_day_demand.csv', index=False)

(21, 8)


In [401]:
sub_df

,CODE,LOCATION,ADDRESS,LATITUDE,LONGITUDE,BRAND,DATE,DEMAND
20,1884,EX KONDAVIL,"KONDAVIL, Cargills Food City, Sri Lanka",9.702180,80.035403,Cargills,2025-03-07,1
26,1164,FC HATTON,"171, Dimbulla Road , Hatton.",6.892798,80.597282,Cargills,2025-03-03,1
27,1316,EX TRINCOMALEE,Linganagar Service station (MPCS),8.576858,81.209883,Cargills,2025-03-03,1
28,1162,FC TRINCOMALEE,"No.190, Thirugnanasampanthan Street, Trincomalee.",8.587364,81.215212,Cargills,2025-03-03,1
29,1288,FC BATTICALOA,"No. 67, St. Anthonys Street, Batticaloa",7.714107,81.696191,Cargills,2025-03-03,1
30,1294,FC AMPARA,"Reegal Cinama , Nidahas Mawatha , Ampara.",7.301756,81.674729,Cargills,2025-03-03,1
32,1425,FC AKKARAIPATHTHU,"No.143/B 1, Kalmunai road, Akkaraipattru.",7.226854,81.850550,Cargills,2025-03-03,1
33,1424,FC KALMUNEI,No. 1034/1 Kalmunai 14,7.414383,81.830633,Cargills,2025-03-03,1
36,1429,FC BATTICALOA 02,"No.03, Gnanasooriyam Square, Trinco Road, Batt...",7.729900,81.674278,Cargills,2025-03-03,1
37,1439,FC POTHUVIL,"Kaleej Front Building”, Main Street, Pothuvil.",6.875271,81.830633,Cargills,2025-03-03,1


In [356]:
import folium
import pandas as pd

def visualize_all_locations(df):
    """
    Visualize all locations from a CSV file on a folium map.
    :param csv_file_path: Path to the CSV file containing location data
    :return: folium.Map object
    """
    # Load the CSV data into a DataFrame
    # try:
    #     df = df
    # except Exception as e:
    #     print(f"Error loading CSV file {csv_file_path}: {e}")
    #     return None
    try:
        # Create a map centered at the mean of all latitudes and longitudes
        map_center = [df['LATITUDE'].mean(), df['LONGITUDE'].mean()]
        m = folium.Map(location=map_center, zoom_start=10)
    except Exception as e:
        print(f"Error getting gps data: {e}")
        return None

    # Add a marker for each location
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            popup=f"CODE: {row['CODE']}<br>LOCATION: {row['LOCATION']}<br>ADDRESS: {row['ADDRESS']}<br>BRAND: {row['BRAND']}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

    # Add title
    title_html = '<h3 align="center" style="font-size:16px">All Locations Map</h3>'
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [357]:
visualize_all_locations(sub_df)

In [358]:
master_gps_df = load_df('../data/master/2/master_gps.csv')

In [359]:
master_gps_df

,CODE,LOCATION,ADDRESS,LATITUDE,LONGITUDE,BRAND
0,3,Wattala SC,"Wattala, Sri Lanka",6.990668,79.893171,Arpico
1,4,Borelasgamuwa SS,"Borelasgamuwa, Sri Lanka",6.840989,79.901719,Arpico
2,5,Hyde Park SC,"Hyde, Sri Lanka",6.917587,79.858519,Arpico
3,10,Nawinna SC,"Nawinna, Sri Lanka",6.853331,79.915072,Arpico
4,105,Matara SC,"Matara, Sri Lanka",5.949631,80.546853,Arpico
...,...,...,...,...,...,...
645,1881,EX ATIGALA,"ATIGALA, Cargills Food City, Sri Lanka",6.894662,80.056286,Cargills
646,1883,EX KUMBALGAMA,"KUMBALGAMA, Cargills Food City, Sri Lanka",6.689882,80.804557,Cargills
647,1884,EX KONDAVIL,"KONDAVIL, Cargills Food City, Sri Lanka",9.702180,80.035403,Cargills
648,1886,EX CAPITAL HEIGHTS,"CAPITAL HEIGHTS, Cargills Food City, Sri Lanka",6.909579,79.900502,Cargills


In [360]:
total_distance = 0
for veh in route_dict.keys():
    print(f"Vehicle id: {veh}")
    path = ''
    for code in route_dict[veh]['route_nodes']:
        if code == '0':
            loc_name = "SMAK"
        else:
            loc = master_gps_df.loc[master_gps_df['CODE']==code]
            if len(loc) is None:
                loc_name = ["SMAK"]
            loc_name = loc['LOCATION'].values[0]
        
        path += f'{loc_name} ({code}) --> '
    path += f"Route Distance: {route_dict[veh]['route_distance']}km (Max: {route_dict[veh]['max_distance_limit']}km) Visits: {route_dict[veh]['num_visits']-1} (Max: {route_dict[veh]['max_visits_limit']})"
    total_distance += route_dict[veh]['route_distance']
    print(path)
    print('\n')
print(f"Total distance for all vehicles: {total_distance}")

Vehicle id: 0
SMAK (0) --> SMAK (0) --> Route Distance: 0km (Max: 400km) Visits: 0 (Max: 10)


Vehicle id: 1
SMAK (0) --> SMAK (0) --> Route Distance: 0km (Max: 400km) Visits: 0 (Max: 10)


Vehicle id: 2
SMAK (0) --> SMAK (0) --> Route Distance: 0km (Max: 400km) Visits: 0 (Max: 10)


Vehicle id: 3
SMAK (0) --> FC GALEWELA (1430) --> FC DAMBULLA 02 (1571) --> FC DAMBULLA (1158) --> EX HABARANA (1381) --> FC IPALOGAMA (1298) --> EX KEKIRAWA (1743) --> EX MADAWALA ULPOTHA (1816) --> EX PALAPATHWELA (1770) --> Matale 2 (SYMX) --> FC MATALE 03 (1659) --> EX POOJAPITIYA (1717) --> SMAK (0) --> Route Distance: 398km (Max: 400km) Visits: 11 (Max: 20)


Vehicle id: 4
SMAK (0) --> FC KARAPITIYA 02 (1651) --> EX KATUGODA (1387) --> EX GALLE FORT (1785) --> EX NELUWA (1724) --> FC GALLE 02 (1444) --> Galle (SFGL) --> FC GALLE (1121) --> Galle 2 (SFGA) --> EX HAPUGALA (1800) --> EX HIKKADUWA 02 (1847) --> EX ARACHCHIKANDA (1809) --> SMAK (0) --> Route Distance: 266km (Max: 400km) Visits: 11 (Max: 2

In [361]:
import pandas as pd

In [362]:
SMAK_KADAWATHA = (7.0038321,79.9394804)

smak_data = {
    "CODE":'0',
    "LOCATION":"SMAK",
    "ADDRESS":"Smak, Kadawatha, Western Province, Sri Lanka",
    "LATITUDE":SMAK_KADAWATHA[0],
    "LONGITUDE":SMAK_KADAWATHA[1]
}


master_df = pd.concat(
    [
        pd.DataFrame(smak_data, index=[0]),
        master_gps_df
    ],
    ignore_index=True
)

In [363]:
map_obj_2 = visualize_routes_per_vehicle(master_df, route_dict, 1)

Processing route for vehicle 0: ['0', '0']
Using cached path for 0 to 0
Plotted path for vehicle 0 with 2 coordinates
Processing route for vehicle 1: ['0', '0']
Using cached path for 0 to 0
Plotted path for vehicle 1 with 2 coordinates
Processing route for vehicle 2: ['0', '0']
Using cached path for 0 to 0
Plotted path for vehicle 2 with 2 coordinates
Processing route for vehicle 3: ['0', '1430', '1571', '1158', '1381', '1298', '1743', '1816', '1770', 'SYMX', '1659', '1717', '0']
Using cached path for 0 to 1430
Using cached path for 1430 to 1571
Using cached path for 1571 to 1158
Using cached path for 1158 to 1381
Using cached path for 1381 to 1298
Using cached path for 1298 to 1743
Fetching path from 1743 ((8.042103, 80.593833)) to 1816 ((7.6155983, 80.6354665))
Path cached for 1743 to 1816: [[8.041093, 80.593668], [8.041006, 80.594209], [8.040992, 80.594296], [8.040951, 80.594457], [8.040897, 80.594966]]... (total 649 points)
Using cached path for 1816 to 1770
Using cached path for 1

In [364]:
map_obj_2[0]

In [365]:
map_obj_2[1]

In [366]:
map_obj_2[2]

In [367]:
map_obj_2[3]

In [368]:
map_obj_2[4]

In [369]:
map_obj_2[5]

In [370]:
map_obj_2[6]

In [371]:
map_obj_2[7]